<a href="https://colab.research.google.com/github/MarkusThill/techdays25/blob/feature-lab1/notebooks/lab1-my-first-onnx-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: Building our first simple ONNX Model

In [ ]:
x = 5
print(x)

5
